In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# Установка seed для воспроизводимости
np.random.seed(42)

n = 500

## 1. Критерий знаков

In [ ]:
def sign_test(x, y):
    signs = np.where(x < y, "-", np.where(x > y, "+", "0"))
    minus_count = np.sum(signs == "-")
    plus_count = np.sum(signs == "+")
    W = min(minus_count, plus_count)
    return W, minus_count, plus_count


### 1.1 выборки

In [ ]:
x1 = np.random.normal(2, np.sqrt(5), n)
y1 = np.random.normal(2, np.sqrt(8), n)
W1, minus1, plus1 = sign_test(x1, y1)
print("1.1:")
print(f"W = {W1}, '-' count = {minus1}, '+' count = {plus1}")

### 1.2 выборки

In [ ]:
# 1.2 выборки
x2 = np.random.normal(2, np.sqrt(5), n)
y2 = np.random.normal(2.2, np.sqrt(7), n)
W2, minus2, plus2 = sign_test(x2, y2)
print("\n1.2:")
print(f"W = {W2}, '-' count = {minus2}, '+' count = {plus2}")


## 2. Ранговый коэффициент корреляции Кэндела

In [ ]:
def kendall_tau(x, y):
    tau, p_value = stats.kendalltau(x, y)
    return tau, p_value

### 2.1 a) Выборку пополам

In [ ]:
x3 = np.random.normal(2, np.sqrt(5), n)
half = n // 2
tau1, p1 = kendall_tau(x3[:half], x3[half:half*2])
print("\n2.1 a):")
print(f"Kendall's tau = {tau1}, p-value = {p1}")


### 2.1 б) Две выборки

In [ ]:
x4 = np.random.normal(2, np.sqrt(5), n)
y4 = 3 * x4 - 5 + np.random.normal(0, 2, n)
tau2, p2 = kendall_tau(x4, y4)
print("\n2.1 б):")
print(f"Kendall's tau = {tau2}, p-value = {p2}")


### 2.2 Низкий коэффициент лин. зависимости

In [ ]:
x5 = np.random.normal(2, np.sqrt(5), n)
y5 = 0.05 * x5 - 5 + np.random.normal(0, 2, n)
tau3, p3 = kendall_tau(x5, y5)
print("\n2.2:")
print(f"Kendall's tau = {tau3}, p-value = {p3}")


## 3. Критерий отсутствия автокорреляции ближайших соседей

In [ ]:
def autocorrelation_test(x):
    autocorr = np.correlate(x[:-1], x[1:], mode='valid')[0] / (np.std(x[:-1]) * np.std(x[1:]) * (len(x)-1))
    return autocorr


### 3.1 Выборка N(3, 7)

In [ ]:
x6 = np.random.normal(3, np.sqrt(7), n)
autocorr1 = autocorrelation_test(x6)
print("\n3.1:")
print(f"Autocorrelation coefficient = {autocorr1}")


### 3.2 Выборка

In [ ]:
x7 = np.random.normal(3, 7, n)

for j in range(1, n):
    x7[j] = x7[j] - x7[j-1] + 2 * np.random.normal(0, 1)
autocorr2 = autocorrelation_test(x7)
print("\n3.2:")
print(f"Autocorrelation coefficient = {autocorr2}")